# Generate bad LLM answers for GSM8K dataset using Gemma-3-1b-it model


In [4]:
import pandas as pd

splits = {'train': 'main/train-00000-of-00001.parquet', 'test': 'main/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/openai/gsm8k/" + splits["train"])
# save to parquet
df.to_parquet("data/gsm8k_train.parquet", index=False)

In [ ]:

# now use ollama gemma3:270m to generate bad llm answers
import os
from google import genai
import pandas as pd

client = genai.Client(api_key=os.getenv("GENAI_API_KEY"))

def generate_response(prompt):
    response = client.models.generate_content(
        model="models/gemma-3-1b-it",
        contents=prompt
    )
    return response.text

# check if gsm8k_with_bad_llm_answers.parquet exists, if yes, load it and continue from there
df = pd.read_parquet("data/gsm8k_train.parquet")
df['bad llm answer'] = ''
if os.path.exists("gsm8k_with_bad_llm_answers.parquet"):
    df_existing = pd.read_parquet("gsm8k_with_bad_llm_answers.parquet")
    # copy the existing answers to the new dataframe
    df['bad llm answer'] = df_existing['bad llm answer']
    print(f"Loaded existing file with {df_existing['bad llm answer'].notnull().sum()} answers")

# iterate over the dataframe and generate bad llm answers for each unanswered question
from tqdm import tqdm
for index, row in tqdm(df.iterrows(), total=len(df)):
    if row['bad llm answer'] != '':
        continue  # skip if already has an answer
    question = row['question']
    response = generate_response(question)
    df.at[index, 'bad llm answer'] = response
# save the dataframe to a new parquet file
df.to_parquet("data/gsm8k_with_bad_llm_answers.parquet")


  0%|          | 8/7473 [00:19<4:57:52,  2.39s/it]


KeyboardInterrupt: 